In [3]:
from transformers import pipeline

pipe = pipeline("text-to-speech", model="suno/bark-small")
text = "Hello, I'm Samantha. I'm an advanced AI designed to be your friend and companion. Please let me know if I can help you with anything!"
output = pipe(text)

/home/optimuseprime/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/optimuseprime/.local/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
/home/optimuseprime/.local/lib/python3.10/site-packages/transformers/models/encodec/modeling_encodec.py:123: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dt

In [3]:
from transformers import AutoProcessor, BarkModel
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"

processor = AutoProcessor.from_pretrained("suno/bark-small")
model = BarkModel.from_pretrained("suno/bark-small", torch_dtype=torch.float16).to(device)
model = model.to_bettertransformer()
model.enable_cpu_offload()

voice_preset = "v2/en_speaker_7"

inputs = processor("Hello, I'm John. I'm an advanced AI designed to be your friend and companion. Please let me know if I can help you with anything! I love life and it gives me great joy to help people. I went to UCLA, the best university in the US, and I loved it there.", voice_preset=voice_preset).to(device)

audio_array = model.generate(**inputs)
audio_array = audio_array.cpu().numpy().squeeze()

/home/optimuseprime/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/optimuseprime/.local/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
/home/optimuseprime/.local/lib/python3.10/site-packages/transformers/models/encodec/modeling_encodec.py:123: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dt

In [4]:
from IPython.display import Audio
sample_rate = model.generation_config.sample_rate
Audio(audio_array, rate=sample_rate)

In [4]:
from IPython.display import Audio
Audio(output["audio"], rate=output["sampling_rate"])

In [7]:
import torch

print(torch.cuda.is_available())

True


In [7]:
import torch
from transformers import VitsTokenizer, VitsModel, set_seed

device = "cuda:0" if torch.cuda.is_available() else "cpu"

tokenizer = VitsTokenizer.from_pretrained("facebook/mms-tts-eng")
model = VitsModel.from_pretrained("facebook/mms-tts-eng")

inputs = tokenizer(text="Hello - my dog is cute", return_tensors="pt")

set_seed(555)  # make deterministic

with torch.no_grad():
   outputs = model(**inputs)

waveform = outputs.waveform[0]

from IPython.display import Audio

Audio(waveform, rate=model.config.sampling_rate)

Some weights of the model checkpoint at facebook/mms-tts-eng were not used when initializing VitsModel: ['flow.flows.0.wavenet.in_layers.0.weight_g', 'flow.flows.0.wavenet.in_layers.0.weight_v', 'flow.flows.0.wavenet.in_layers.1.weight_g', 'flow.flows.0.wavenet.in_layers.1.weight_v', 'flow.flows.0.wavenet.in_layers.2.weight_g', 'flow.flows.0.wavenet.in_layers.2.weight_v', 'flow.flows.0.wavenet.in_layers.3.weight_g', 'flow.flows.0.wavenet.in_layers.3.weight_v', 'flow.flows.0.wavenet.res_skip_layers.0.weight_g', 'flow.flows.0.wavenet.res_skip_layers.0.weight_v', 'flow.flows.0.wavenet.res_skip_layers.1.weight_g', 'flow.flows.0.wavenet.res_skip_layers.1.weight_v', 'flow.flows.0.wavenet.res_skip_layers.2.weight_g', 'flow.flows.0.wavenet.res_skip_layers.2.weight_v', 'flow.flows.0.wavenet.res_skip_layers.3.weight_g', 'flow.flows.0.wavenet.res_skip_layers.3.weight_v', 'flow.flows.1.wavenet.in_layers.0.weight_g', 'flow.flows.1.wavenet.in_layers.0.weight_v', 'flow.flows.1.wavenet.in_layers.1.wei

In [8]:
from transformers import pipeline
from datasets import load_dataset

synthesiser = pipeline("text-to-speech", "microsoft/speecht5_tts")

embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embedding = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)
# You can replace this embedding with your own as well.

speech = synthesiser("Hello, my dog is cooler than you!", forward_params={"speaker_embeddings": speaker_embedding})

from IPython.display import Audio

Audio(speech["audio"], rate=speech["sampling_rate"])

/home/optimuseprime/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/585M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/232 [00:00<?, ?B/s]

spm_char.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/50.7M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/7931 [00:00<?, ? examples/s]

In [8]:
from RealtimeTTS import TextToAudioStream, SystemEngine, CoquiEngine

engine = CoquiEngine() # replace with your TTS engine
stream = TextToAudioStream(engine)
stream.feed("Imagine a stage, lit by the warmth of a setting sun. On it stands a person, their voice strong yet tender, filling the air with words that resonate and empower. They speak of the endless possibilities that lie within each individual, igniting a spark in every heart.")
stream.play(output_wavfile=stream.engine.engine_name + "_output.wav")

 > Using model: xtts


ERROR:root:Error initializing main coqui engine model: CUDA out of memory. Tried to allocate 26.00 MiB. GPU 
Traceback (most recent call last):
  File "/home/optimuseprime/Projects/samantha/venv/lib/python3.10/site-packages/RealtimeTTS/engines/coqui_engine.py", line 509, in _synthesize_worker
    tts = load_model(checkpoint, tts)
  File "/home/optimuseprime/Projects/samantha/venv/lib/python3.10/site-packages/RealtimeTTS/engines/coqui_engine.py", line 493, in load_model
    tts.to(device)
  File "/home/optimuseprime/Projects/samantha/venv/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1173, in to
    return self._apply(convert)
  File "/home/optimuseprime/Projects/samantha/venv/lib/python3.10/site-packages/torch/nn/modules/module.py", line 779, in _apply
    module._apply(fn)
  File "/home/optimuseprime/Projects/samantha/venv/lib/python3.10/site-packages/torch/nn/modules/module.py", line 779, in _apply
    module._apply(fn)
  File "/home/optimuseprime/Projects/samantha/v

Error loading model for checkpoint /home/optimuseprime/Projects/samantha/models/v2.0.2: CUDA out of memory. Tried to allocate 26.00 MiB. GPU 


KeyboardInterrupt: 